## Story Cloze: Bert for Sequence Classification

#### install tools

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datasets
from datasets import load_dataset, load_metric
from datasets import Dataset
import ast


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [105]:
# load dataset
dataset = pd.read_csv('/content/drive/MyDrive/Project/datasets/concat_2017_labeled.csv').drop('Unnamed: 0',axis=1)

In [106]:
dataset.cands = dataset.cands.apply(ast.literal_eval)
data = dataset.explode('cands').reset_index(drop=True)
data['label'] = data.apply(lambda row: 1 if row['answer'] == row['cands'] else 0, axis=1)

In [107]:
data.head(1)

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,answer,cands,story,label
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",His weight continued to increase and he develo...,David noticed he had put on a lot of weight re...,0


### Modify Training Test and Validation Datasets

In [108]:
def modify_training(data):
  new = pd.DataFrame()
  new['stories'] = data.story
  new['endings'] = data.answer
  new['label'] = data.label
  return new.sample(frac=1,replace=False).reset_index(drop=True)

In [109]:
data_mod = modify_training(data)
train_df = data_mod[:70000]
validation_df = data_mod[70000:85000]
test_df = data_mod[85000:]

In [110]:
print('Training Labels:', train_df.label.value_counts())
print('Validation Labels:', validation_df.label.value_counts())
print('Test Labels:', test_df.label.value_counts())

Training Labels: label
1    35054
0    34946
Name: count, dtype: int64
Validation Labels: label
0    7588
1    7412
Name: count, dtype: int64
Test Labels: label
1    10199
0    10129
Name: count, dtype: int64


In [111]:
#"Looks pretty even to me!"

In [112]:
print('Lets try to find the max length of sequence')
print(' ')
print('Maximum Num Training Words:', max([len(x.split()) for x in train_df.stories]) + max([len(x.split()) for x in train_df.endings]) )

Lets try to find the max length of sequence
 
Maximum Num Training Words: 81


In [113]:
### turn all train, validation, and test datasets into a dataset dictionary ###
train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(validation_df)
test_dataset = Dataset.from_dict(test_df)
data_dict = datasets.DatasetDict({"train":train_dataset,
                                        "validation" : val_dataset,
                                        "test":test_dataset})

## Look at some examples

In [114]:
def show_one(example):
    print(f"Story: {example['stories']}")
    print(f"\nEnding: {example['endings']}")
    print(f"\nLabel: {example['label']}")

In [115]:
show_one(data_dict['train'][999])

Story: Richard was employed as a fundraiser for a large non-profit company. The company's mission was to provide food to needy families. Richard's job was easy during the holidays when many felt generous. Richard's job was hard during the summer when many felt indulgent.

Ending: Richard soldiered on year-round, believing his company to be noble.

Label: 0


In [116]:
show_one(data_dict['validation'][999])

Story: Max loved technology. He got a new computer. It had all the bells and whistles. He installed some games on it and started playing.

Ending: The computer was so much fun!

Label: 1


## Preprocess

In [117]:
### load tokenizer ###
from transformers import AutoTokenizer
checkpoint = '/content/drive/MyDrive/Project/bert_model_weights'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#### Tokenize each sentence

In [118]:
MAX_SEQUENCE_LENGTH = 85
def preprocess_function(examples):
    return tokenizer(examples['stories'], examples['endings'], max_length=MAX_SEQUENCE_LENGTH,
                     truncation=True, padding='max_length',return_tensors='tf')

In [119]:
pre_tokenizer_columns = set(data_dict["train"].features)
encoded_dataset = data_dict.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20328 [00:00<?, ? examples/s]

Columns added by tokenizer: ['token_type_ids', 'attention_mask', 'input_ids']


#### Look at some tokenization and preprocessing outputs

In [120]:
examples = data_dict["train"][:3]
features = preprocess_function(examples)
print(
    len(features["input_ids"]),
    len(features["input_ids"][0]),
    [len(x) for x in features["input_ids"]],
)

3 85 [85, 85, 85]


In [121]:
[tokenizer.decode(features["input_ids"][x]) for x in range(len(features))]

['[CLS] when i got home from work, my house reeked. i could not figure out what the awful smell was. i looked everywhere. i finally checked my gym bag. [SEP] my best friend has thrown her smelly socks in it. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] tom was helping his brother tear down a shed. they threw all the discarded wood into a pile. tom and his brother wanted to move the pile into the back of his yard. they loaded all the wood onto a trailer. [SEP] tom and his brother towed the wood in the trailer. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] we went to this fun summer camp as kids. they had horseback riding and baseball and water sports. we would play all day and have so 

## Load Model

In [122]:
from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

model = TFBertForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [124]:
model.config.dropout = 0.3
model.config.hidden_dropout_prob = 0.3
model.config.attention_probs_dropout_prob = 0.3

In [125]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

tf_test_dataset = model.prepare_tf_dataset(
    encoded_dataset["test"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

Evaluation Metrics

In [126]:
import evaluate
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")

In [127]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    rec = recall.compute(predictions=predictions, references=labels)['recall']

    return {"accuracy": acc, "recall": rec}

In [132]:
from transformers import create_optimizer

num_epochs = 3
batch_size = 16
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=3e-05, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [133]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [134]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [135]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=[metric_callback],
)

Epoch 1/3
4375/4375 [==============================] - 1564s 345ms/step - loss: 0.6981 - val_loss: 0.6931 - accuracy: 0.5059 - recall: 0.0000e+00
Epoch 2/3
4375/4375 [==============================] - 1484s 339ms/step - loss: 0.6940 - val_loss: 0.6938 - accuracy: 0.4941 - recall: 1.0000
Epoch 3/3
4375/4375 [==============================] - 1531s 350ms/step - loss: 0.6935 - val_loss: 0.6934 - accuracy: 0.4941 - recall: 1.0000


In [138]:
predictions = model.predict(tf_test_dataset)

1271/1271 [==============================] - 132s 104ms/step


In [142]:
outputs = predictions.logits
classifications = np.argmax(outputs, axis=1)

In [152]:
classifications.mean()

1.0

In [153]:
# Baseline Sequnce2sequence model predicts evreything to be a correct label